# Final Project: Baseline CV Models for Segmentation of PV Solar Panels in Remote Sensing Imagery

### Alejandro Vega Nogales
#### May 2025

## Introduction
As part of the final project for the "Introduction to Computer Vision" course, we used the Maxar 30cm and 15cm Maxar imagery datasets presented in "A solar panel dataset of very high resolution satellite imagery to support the Sustainable Development Goals" to train several of the segmentation architectures presented in the course and different combination of "backbone" encoders using [segmentation-models-pytorch](https://smp.readthedocs.io/en/latest/quickstart.html) and [torch lightning](https://lightning.ai/lightning-ai/studios/image-segmentation-with-pytorch-lightning?view=public&section=featured) to quickly iterate over the different models and hyperparameters. The goal of this project is to provide a baseline for future work in the field of remote sensing imagery segmentation, specifically for the detection of solar panels in high-resolution satellite images. We explore the use of data augmentations such as random rotations, flips, and color jittering using the [albumentations](https://albumentations.ai/docs/) library, as well as using two different loss functions: [Binary Cross Entropy (BCE)](https://smp.readthedocs.io/en/latest/losses.html#softbcewithlogitsloss) and [Dice Loss](https://smp.readthedocs.io/en/latest/losses.html#diceloss). We use the [mean Intersection over Union (mIoU)](https://lightning.ai/docs/torchmetrics/stable/detection/intersection_over_union.html) metric to evaluate the performance of the models on pixel-wise segmentation of solar panels. 

## Brief Dataset Description and Limitations 

<figure style="width: 40%; margin: auto;">
    <img src="report/assets/figures/dataset_masks_example.png" alt="dataset masks example" style="width: 100%;">
    <figcaption>Example of dataset image chip, masks of bounding boxes, and the mask applied to the image chip</figcaption>
</figure>

The Maxar dataset from [Worldview-3](https://www.eoportal.org/satellite-missions/worldview-3) covers 3 tiles of a satellite image strip over Southern Germany and is freely available [at this link](https://resources.maxar.com/geospatial-foundation/15-cm-hd-and-30-cm-view-ready-solar-panels-germany). It includes 2,542 .tif image chips and the same amount of corresponding .txt YOLO format labels. Each .txt file contains 1 or more lines with the class being the confidence of the human annotator. There are some inherent limitations to our task when using this dataset:
1. dataset is designed for object detection, not segmentation. Therefore our labels include plenty of non-solar panel pixels in each label. 
2. dataset is RGB only with no multispectral bands (see next steps for importance)
3. dataset contains no negative samples (i.e. no images without solar panels) increasing the risk of false positives when evaluating the model
4. dataset is mostly from rooftop solar panel installations, which may not be representative of other types of installations (e.g. ground-mounted solar farms) and will not generalize well to other regions of the world

<!-- show example of problems *single-class* OD labels presents in segmentation -->
#### 3rd and 4th row are examples of problems single-class OD labels present in segmentation 

<figure style="width: 40%; margin: auto;">
    <img src="report/assets/figures/train_dataloader_augm_bad_labels_example.png" alt="3rd and 4th row are examples of problems single-class OD labels presents in segmentation" style="width: 100%;">
    <figcaption>Image chips with albumentations transforms applied and binary masks sampled from Train Dataloader</figcaption>
</figure>

## Work and Training Methodology

See details of Torch Lightning implementation in our [training notebook](https://github.com/avega17/CCOM_MS_Spring_2025_EO_PV_research/blob/main/%5BColab_train%5D%20pv_segm_smp_baselines.ipynb). If you clone [our repo]https://github.com/avega17/CCOM_MS_Spring_2025_EO_PV_research) and download the dataset to the expected folder structure, you can run the training notebook and reproduce our results in a Google Colab environment. 

<a target="_blank" href="https://colab.research.google.com/github/avega17/CCOM_MS_Spring_2025_EO_PV_research/blob/main/%5BColab_train%5D%20pv_segm_smp_baselines.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

Models were trained using Google's [Colab Pro+ subscription](https://colab.research.google.com/signup) primarily with the L4 and A100 GPUs to enable larger batch sizes and faster training. 

We followed some of the encoder recommendations from the [segmentation models pytorch documentation](https://smp.readthedocs.io/en/latest/encoders.html#choosing-the-right-encoder). 

## Results

- Several model architectures and encoder combinations caused NaN loss during training. 
- Pyramid Attention Network (PAN), DeepLabV3+, and plain UNet with ResNet101 encoders showed consistent performances
- Segformer with the MIT (mixed vision transformer) encoder showed the best performance reaching ~0.89 for the validation IoU but was slower to train. We also tried the recently added []
- PSPNet (Pyramid Scene Parsing Network) and MANet (Multi-Attention Network) had consistently poor performance with the encoders we tried (ResNet101, 
- Bigger dataset is needed to avoid overfitting and improve generalization to other regions of the world
- The experiments carried out here with torch lightning, segmentation models pytorch, and albumentations with a small, manageable dataset, has established the groundwork for future work in training pipelines for segmentation of solar panels in remote sensing imagery.

<div style="display: flex; flex-direction: row; justify-content: space-between;">
    <div style="width: 48%;">
        <figure style="width: 100%; margin: auto;">
            <img src="report/assets/figures/eval_segformer_mitb5_hd_512.png" alt="eval_segformer_mitb5_hd_512.png" style="width: 90%;">
            <figcaption>Segformer with MITB5 encoder, and sample of inference over validation set</figcaption>
        </figure>
    </div>
    <div style="width: 48%;">
        <figure style="width: 100%; margin: auto;">
            <img src="report/assets/figures/model_eval_PAN_efficientnet-b5 (hd)_49E.png" alt="model_eval_PAN_efficientnet-b5 (hd)_49E.png" style="width: 90%;">
            <figcaption>PAN with EfficientNetB5 encoder, and sample of inference over validation set</figcaption>
        </figure>
    </div>
</div>

Best performing model checkpoint were saved to wandb artifacts and can be downloaded from the wandb project page. 
Below we add a link to our barebones report on Wandb where we logged the training runs and hyperparameters. We include some sample inference images for some of the later models (this was enabled later after training some of the initial models):

In [4]:
# import iPython iFrame display
from IPython.display import IFrame
IFrame('https://api.wandb.ai/links/avega/69vficed', width=1200, height=800)

# Future Work and Next Steps
- Identify and prepare proper test set which model has not seen to evaluate how well it generalizes to unseen data.
- Use multispectral bands (e.g. NIR, SWIR) to improve segmentation accuracy. For fast iteration, I will initially use the provided Sentinel2 10m 13-band imagery global dataset in [GloSoFarID](https://arxiv.org/abs/2404.05180)
- As we learned in class, applying filters that highlight the features we want out model to learn can significantly improve performance. One way to do this in remote sensing is via Spectral Index both for Solar Panels and relevant surrounding (vegetation, urban, etc). Implement the use of the 3 Photovoltaic Spectral Indices (simply algebra with the required bands) listed in ["Developing a Universal Spectral Index for Solar Photovoltaic Panels: A Methodology for Spatial Information Extraction from Satellite Remote Sensing Imagery"](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=4855811) to augment the RGB bands with meaningful multispectral features without needing to add so many multispectral channels 
- Look into using Google Cloud Credits from course for Google's [Solar API](https://developers.google.com/maps/documentation/solar/coverage) which offers [data layers](https://developers.google.com/maps/documentation/solar/data-layers) with RGB aerial imagery, Digital Surface Model (DSM), Solar Flux statistics for the area, and more.
- Set up data pipeline to use [STAC Catalogs](https://stacspec.org/en/tutorials/intro-to-stac/) to access and query multispectral satellite imagery from different sources so we can start using our [consolidated PV dataset](https://github.com/avega17/CCOM_MS_Spring_2025_EO_PV_research/blob/main/fetch-pv-datasets-ESDA.ipynb) 
- Use [Segment Anything Model 2 (SAM2)](https://docs.ultralytics.com/models/sam-2/#how-does-sam-2-handle-occlusions-and-object-reappearances-in-video-segmentation) or segmentation-capable [YOLO models from ultralytics](https://docs.ultralytics.com/tasks/segment/) for unsupervised ways to improve our segmentation mask labels from the Polygon bounding box labels we have collected